## 0) Imports

In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline

from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.ar_model import ar_select_order
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

# from cleanup import clean_data
# from engineer import feature_engineer

plt.rcParams['figure.figsize'] = (12, 8)

## 1) Business Goal

Build a model that can predict tomorrows temperature, given the temprature until today, as precisely as possible.

## 2) Get the Data

### 2.1) Load the Data

In [ ]:
def get_temp_data():
    """
    read temperature data
    """
    return pd.read_csv(
        './data/TG_STAID002759.txt', 
        parse_dates=True, 
        sep=',', 
        skiprows=19, 
        index_col=1
    )

### 2.2) Clean the Data

In [ ]:
def get_clean_data(year=1950):
    df = get_temp_data()

    df.columns = [col.strip() for col in df.columns]
    df.index.freq = "D" # TODO: try W

    df.rename(columns={'TG':'temp'}, inplace=True)
    df.index.names = ['date']

    return df[df.index.year > year][['temp']].copy()

df = get_clean_data()
df

In [ ]:
## 3) Train-Test-Split 
# TODO: remove and update future numeration

In [ ]:
# df_train = df[:-365].copy()
# df_test = df[-365:].copy()

## 4) Visualize the Data

### 4.1) Actual temperature

In [ ]:
px.line(df, y="temp").show()

### 4.2) Trend

In [ ]:
def get_trend(df):
    """ build and return trend """
    timestep = pd.DataFrame(range(len(df)))
    
    X = pd.DataFrame(range(len(df)))
    y = df['temp']
    
    m = LinearRegression()
    m.fit(X, y)

    return m.predict(X)

trend = get_trend(df)

In [ ]:
fig = go.Figure()
# fig.add_trace(go.Scatter(x=df.index.values, y=df['temp'], name='Temperature'))
fig.add_trace(go.Scatter(x=df.index.values, y=trend, name='Trend'))

fig.show() # Looks like global warming is not an anti-globalization fantasy 😕 

## 4.3) Seasonality

In [ ]:
df.groupby(df.index.month)['temp'].mean().plot.bar()

In [ ]:
def get_seasonality_ohe(df):
    return pd.get_dummies(df.index.month, drop_first=True, prefix='month').set_index(df.index)

def get_seasonal_trend(df):
    X = get_seasonality_ohe(df)
    X['timestep'] = range(len(df))
    
    y = df['temp']
    m = LinearRegression()
    m.fit(X, y)

    return m.predict(X) # it includes overall trend

seasonal_trend = get_seasonal_trend(df)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df.index.values, y=df['temp'], name='Temperature'))
fig.add_trace(go.Scatter(x=df.index.values, y=seasonal_trend, name='Seasonal trend'))
fig.add_trace(go.Scatter(x=df.index, y=trend, name='Trend'))

fig.show()

### 4.4) Remainder

In [ ]:
remainder = df['temp'].values - seasonal_trend

In [ ]:
fig = px.scatter(df, x = df.index, y=remainder, trendline="ols")
fig.update_traces(showlegend=True)
fig.show()

In [ ]:
remainder.std()

### 4.5) Autoregression

In [ ]:
def add_lags(df, n=3):
    for i in range(1,n+1):
        df[f"lag{i}"] = df['remainder'].shift(i)
        
    tmp.dropna()
        
    return df

tmp = add_lags(pd.DataFrame(remainder, columns=['remainder']))

corr = tmp.corr()
corr

In [ ]:
import seaborn as sns

sns.heatmap(corr)

In [ ]:
melted = tmp.melt(id_vars=['remainder'])

traces = px.scatter(melted, x="value", y="remainder", color='variable', trendline="ols")

fig = go.Figure()
fig.add_trace(traces.data[1])
fig.add_trace(traces.data[3])
fig.add_trace(traces.data[5])
fig.update_traces(showlegend=True)
fig.show()

# Note: this plot shows us correlation between previous value (lagN) and current remainder-value. 
# The strongest correlation is for lag1. Meaning that yesterday's weather has the biggest impact to today's weather.

In [ ]:
# correnation between remainder and lags (X is a lag level, y is correlation)
# lag0 is correlates to remainder with corr=1 (because there is no lag and lag0=remainder)

print(plot_acf(remainder))

In [ ]:
# partial (direct) correlation between lag level and remainder (that is not explained by previous lag with lower level)
print(plot_pacf(remainder, method='ywm'))

In [ ]:
def get_ar_lags(remainder):
    """ number of lags (previous days) that should be used by autoregression model """
    selected_order = ar_select_order(remainder, maxlag=10)

    return selected_order.ar_lags

get_ar_lags(remainder)

In [ ]:
def get_remainder_ar(remainder):
    """ calculate remainder parts that could be deduced from previous days """
    
    model_ar = AutoReg(endog=remainder, lags=len(get_ar_lags(remainder))).fit() # TODO: check if number of legs can be taken from selected_order directly

    return model_ar.predict()

remainder_ar = get_remainder_ar(remainder)

In [ ]:
tmp = pd.DataFrame({
    'temp': df['temp'],
    'explained_by_ar_and_seasonality': remainder_ar + seasonal_trend,
    'date': df.index
})

tmp = tmp.dropna().melt(id_vars=['date'])

line = px.line(tmp[pd.to_datetime(tmp['date']).dt.year > 2020], x="date", y="value", color='variable')
line.show() # feel free to zoom plot in

In [ ]:
tmp = pd.DataFrame({
    'remainder': remainder,
    'remainder_ar': remainder_ar,
    'noise': remainder - remainder_ar,
    'date': df.index
})

tmp = tmp.dropna().melt(id_vars=['date'])

line = px.line(tmp[pd.to_datetime(tmp['date']).dt.year > 2020], x="date", y="value", color='variable')

line.show() # feel free to zoom plot in

#### Standart deviation

In [ ]:
tmp = pd.DataFrame({
    'temp': df['temp'],
    'seasonal_trend': seasonal_trend,
    'remainder': remainder,
    'noise': remainder - remainder_ar
})
tmp = tmp.dropna()
tmp.std()

## 5) Feature Engineering

In [ ]:
# TODO: does it work if you remove return statements?

# def add_trend(df):
#     df['trend'] = get_trend(df)
    
#     return df

def add_timestep(df):
    
#     print('add_timestep', df.shape)
    
    df['timestep'] = range(len(df))
    
    return df

# def add_seasonality_ohe(df):
#     print('add_seasonality_ohe', df.shape)
#     return df.join(get_seasonality_ohe(df))

def add_seasonal_trend_and_ohe(df):

#     print('add_seasonal_trend_and_ohe', df.shape)

    df['seasonal_trend'] = get_seasonal_trend(df)

    return df.join(get_seasonality_ohe(df))

def add_remainder(df):
    
#     print('add_remainder', df.shape)
#     print(df)
    
    df['remainder'] = df['temp'] - df['seasonal_trend'] #TODO: does it work without .values ?
    
    return df

# def add_lags(df):
#     df['lag1'] = df['remainder'].shift(1)
#     df['lag2'] = df['remainder'].shift(2)
#     df['lag3'] = df['remainder'].shift(3)

#     df.dropna()
    
#     return df

# def add_remainder_ar(df):
#     df['remainder_ar'] = get_remainder_ar(df['remainder'])
    
#     return df

def dropna(df):
    
#     print('dropna', df.shape)
    
    return df.dropna()

# dropna(add_remainder(add_seasonal_trend_and_ohe(add_timestep(df))))

# # this pipeline can be used for train and test data (it does not use "temp" column)
# pipeline_without_temp = make_pipeline(
#     FunctionTransformer(add_timestep),
#     FunctionTransformer(add_seasonal_trend_and_ohe),
# #     FunctionTransformer(add_remainder_ar),
#     FunctionTransformer(dropna), # TODO: can it be done more python'ish?
# )

def drop_remainder_and_seasonal_trend(df):
    return df.drop(columns=['seasonal_trend', 'remainder'])

pipeline = make_pipeline(
    FunctionTransformer(add_timestep),
    FunctionTransformer(add_seasonal_trend_and_ohe),
    FunctionTransformer(add_remainder),
    FunctionTransformer(add_lags),
    FunctionTransformer(dropna), # TODO: can it be done more python'ish?
    FunctionTransformer(drop_remainder_and_seasonal_trend), # TODO: can it be done more python'ish?
)

In [ ]:
df = get_clean_data()

df_train = df[:-365].copy()
df_test = df[-365:].copy()

df_train_fe = pipeline.transform(df_train)

X_train = df_train_fe.drop(columns=['temp']).copy()
y_train = df_train_fe['temp'].copy()

# X_train

## 6) Train a model

In [ ]:
m = LinearRegression()
m.fit(X_train, y_train)
m.score(X_train, y_train) # makes not much sense because list of features was built based on "temp" column (but still interesting to see the score)

## 7) Cross-Validate and Optimize Hyperparameters

## 8) Test

In [ ]:
df_test_fe = feature_engineer(df_test)

y_test = df_test_fe.copy().iloc[:,0]
X_test = df_test_fe.copy().iloc[:,1:]

In [ ]:
r2 = round(m.score(X_test, y_test), 2)

In [ ]:
print(f'The R-squared of our model is {r2}')